In [3]:
import sqlite3

path = ''
db_name = 'repository.db'

try:
        
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)カーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE rep (name TEXT, language TEXT, star INT);'

    # SQLの実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

In [ ]:
# ライブラリを読み込む
import requests
from bs4 import BeautifulSoup
import time

# 結果を保存するリスト
repositories = []

# Google組織のリポジトリ一覧ページ
base_url = "https://github.com/orgs/google/repositories"
print(f"アクセス先: {base_url}")

# より詳細なヘッダーを設定
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Cache-Control': 'max-age=0'
}

# セッションを使用
session = requests.Session()
session.headers.update(headers)

# ページ番号を変えながらリポジトリを取得する
page = 1
max_pages = 94  

while page <= max_pages:
    # base_url(https://github.com/orgs/google/repositories)からページを変更しながら全ページを訪れる
    if page == 1:
        url = f"{base_url}?tab=repositories"
    else:
        url = f"{base_url}?page={page}&tab=repositories"
    
    print(f"\n[ページ {page}] {url}")
    
    try:
        # ページにアクセス
        res = session.get(url, timeout=15)
        
        print(f"  ステータスコード: {res.status_code}")
        
        if res.status_code == 503:
            print("  503エラー")
            time.sleep(5)
            # リトライ
            res = session.get(url, timeout=15)
            print(f"  再試行後のステータスコード: {res.status_code}")
        
        if res.status_code != 200:
            print(f"  アクセスできませんでした。")
            break
        
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.content, 'html.parser')
        
        # リポジトリのリスト項目を取得
        repo_items = soup.find_all('li', class_='ListItem-module__listItem--k4eMk')
        
        # 別のクラス名も試す
        if not repo_items:
            repo_items = soup.find_all('li', {'data-testid': lambda x: x and 'repository' in str(x).lower()})
        
        if not repo_items:
            # より広い範囲で検索
            repo_items = soup.find_all('li')
            # リポジトリ名を含むものだけフィルター
            repo_items = [item for item in repo_items if item.find('a', href=lambda x: x and '/google/' in str(x))]
        
        if not repo_items:
            print("  リポジトリが見つかりませんでした")
            # HTMLの一部を表示してデバッグ
            print("  HTMLの最初の500文字:")
            print(soup.text[:500])
            break
        
        print(f"  ✓ {len(repo_items)}個のリポジトリ要素を発見")
        
        for item in repo_items:
            # リポジトリ名を取得
            title_h4 = item.find('h4', class_='Title-module__heading--s7YnL')
            if title_h4:
                repo_link = title_h4.find('a', class_='Title-module__anchor--GmXUE')
                if repo_link:
                    repo_name = repo_link.get_text().strip()
                else:
                    continue
            else:
                # 別の方法で探す
                repo_link = item.find('a', href=lambda x: x and x.startswith('/google/'))
                if repo_link:
                    repo_name = repo_link.get_text().strip()
                else:
                    continue
            
            # 言語を取得（いろんな方法を試す）
            language = "不明"
            
            # ReposListItem-module__Text_4--mkG7R クラス（検証より)
            language_span = item.find('span', class_='ReposListItem-module__Text_4--mkG7R')
            if language_span:
                language = language_span.get_text().strip()
            else:
                # itemprop="programmingLanguage"
                language_span = item.find('span', {'itemprop': 'programmingLanguage'})
                if language_span:
                    language = language_span.get_text().strip()
                else:
                    # LanguageCircle の隣のspan
                    lang_circle = item.find('div', class_=lambda x: x and 'LanguageCircle' in str(x))
                    if lang_circle and lang_circle.parent:
                        next_span = lang_circle.find_next_sibling('span')
                        if next_span:
                            language = next_span.get_text().strip()
            
            # スター数を取得（複数の方法を試す）
            star_count = "0"
            
            # aria-labelに"star"を含むaタグ
            star_link = item.find('a', {'aria-label': lambda x: x and 'star' in str(x).lower()})
            if star_link:
                aria_label = star_link.get('aria-label', '')
                # "34 stars" や "1,234 stars" から数字を抽出
                star_count = aria_label.split()[0].replace(',', '')
            else:
                # stargazers へのリンクを探す
                star_link = item.find('a', href=lambda x: x and 'stargazers' in str(x))
                if star_link:
                    # リンクのテキストから数字を取得
                    star_text = star_link.get_text().strip()
                    # カンマを削除
                    star_count = star_text.replace(',', '')
                else:
                    # octicon-star アイコンの隣のテキスト
                    star_svg = item.find('svg', class_='octicon-star')
                    if star_svg and star_svg.parent:
                        # 親要素のテキストからアイコン以外の部分を取得
                        star_text = star_svg.parent.get_text().strip()
                        # 数字だけ抽出
                        import re
                        match = re.search(r'[\d,]+', star_text)
                        if match:
                            star_count = match.group().replace(',', '')
            
            # 結果を保存
            repo_info = {
                'リポジトリ名': repo_name,
                '主要な言語': language,
                'スター数': star_count
            }
            repositories.append(repo_info)
            print(f"    ✓ {repo_name} | {language} | {star_count}")
        
        if len([item for item in repo_items if item.find('a', href=lambda x: x and '/google/' in str(x))]) == 0:
            print(" このページではリポジトリ情報を抽出できませんでした")
        
        # 次のページへ
        page += 1
        
        # サーバーに負荷をかけないため待機
        time.sleep(2)
        
    except Exception as e:
        print(f"  → エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
        break

# 結果を表示
print("\n" + "="*60)
print(f"取得したリポジトリ数: {len(repositories)}")
print("="*60)

for i, repo in enumerate(repositories[:20], 1):  
    print(f"{i}. {repo['リポジトリ名']}")
    print(f"   言語: {repo['主要な言語']}")
    print(f"   スター: {repo['スター数']}")
    print()

アクセス先: https://github.com/orgs/google/repositories

[ページ 1] https://github.com/orgs/google/repositories?tab=repositories
  ステータスコード: 200
  ✓ 30個のリポジトリ要素を発見
    ✓ bazel-common | Starlark | 91
    ✓ XNNPACK | C | 2.2k
    ✓ osv-scalibr | Go | 536
    ✓ open-dice | C++ | 26
    ✓ heir | C++ | 609
    ✓ dive | C++ | 17
    ✓ chromium-policy-vulnfeed | Go | 7
    ✓ site-kit-wp | JavaScript | 1.3k
    ✓ toucan | C++ | 48
    ✓ xls | C++ | 1.4k
    ✓ nomulus | Java | 1.8k
    ✓ oss-fuzz | Shell | 12k
    ✓ docsy | JavaScript | 2.9k
    ✓ tunix | Python | 1.9k
    ✓ gemma.cpp | C++ | 6.6k
    ✓ osv-scanner | Go | 8.1k
    ✓ cassowary.dart | Dart | 44
    ✓ nsjail | C++ | 3.6k
    ✓ zerocopy | Rust | 2.1k
    ✓ flatbuffers | C++ | 25k
    ✓ dwh-migration-tools | Java | 54
    ✓ android-cuttlefish | C++ | 571
    ✓ ink | C++ | 49
    ✓ or-tools | C++ | 13k
    ✓ earthengine-catalog | Jsonnet | 121
    ✓ error-prone | Java | 7.1k
    ✓ angle | C++ | 3.8k
    ✓ perfetto | C++ | 5k
    ✓ oss-fuzz-v

In [ ]:
# SQLiteデータベースに保存
import sqlite3

print("データベースへの保存を開始します")

# データベースに接続
conn = sqlite3.connect('repository.db')
cursor = conn.cursor()

# データを挿入
insert_count = 0
for repo in repositories:
    try:
        cursor.execute('''
            INSERT INTO rep (name, language, star)
            VALUES (?, ?, ?)
        ''', (repo['リポジトリ名'], repo['主要な言語'], repo['スター数']))
        insert_count += 1
    except sqlite3.Error as e:
        print(f"挿入エラー: {repo['リポジトリ名']} - {e}")

conn.commit()
print(f"✓ {insert_count}件のデータを挿入しました")

# 確認
cursor.execute('SELECT COUNT(*) FROM rep')
total_count = cursor.fetchone()[0]
print(f"✓ データベース内の総レコード数: {total_count}件")

conn.close()
print("✓ データベース接続を閉じました")

データベースへの保存を開始します
✓ 2812件のデータを挿入しました
✓ データベース内の総レコード数: 2812件
✓ データベース接続を閉じました


In [26]:
path = ''
db_name = 'repository.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)カーソルオブジェクトを取得
    cur = conn.cursor()

    # データを参照するSQL
    # SELECT * FROM テーブル名;
    # *の部分は全てのカラムを意味する
    sql = 'SELECT * FROM rep;'

    # SQLの実行
    cur.execute(sql)

    # 変更を保存する
    # conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

else:
    for row in cur:
        # 行データ(row)はタプルなので、アンパックして列データを取得
        language, name, star = row
        print(name, language, star)

finally:
    # DBへの接続を閉じる
    conn.close()

Starlark bazel-common 91
C XNNPACK 2.2k
Go osv-scalibr 536
C++ open-dice 26
C++ heir 609
C++ dive 17
Go chromium-policy-vulnfeed 7
JavaScript site-kit-wp 1.3k
C++ toucan 48
C++ xls 1.4k
Java nomulus 1.8k
Shell oss-fuzz 12k
JavaScript docsy 2.9k
Python tunix 1.9k
C++ gemma.cpp 6.6k
Go osv-scanner 8.1k
Dart cassowary.dart 44
C++ nsjail 3.6k
Rust zerocopy 2.1k
C++ flatbuffers 25k
Java dwh-migration-tools 54
C++ android-cuttlefish 571
C++ ink 49
C++ or-tools 13k
Jsonnet earthengine-catalog 121
Java error-prone 7.1k
C++ angle 3.8k
C++ perfetto 5k
Python oss-fuzz-vulns 166
C++ skia 10k
Go skia-buildbot 158
Java device-infra 58
Python meridian 1.2k
Rust gn-language-server 18
Rust matr 9
C++ dawn 778
Python langfun 875
Go osv-scanner-v1 2
HTML adk-docs 974
C++ quiche 801
Kotlin private-compute-libraries 34
Java gson 24k
C kafel 340
Python adk-python 16k
Java mug 444
C++ biva 6
Kotlin ground-android 272
Go gvisor 17k
TypeScript budoux-extension 16
C++ tabuli 35
C++ filament 19k
Go adk-go 4.6k
P